# Построить модель классификации: Вкусные азиатские и индийские кухни


## Классификаторы кухни 2

Во втором уроке по классификации мы изучим `дополнительные способы` классификации категориальных данных. Также мы разберем последствия выбора одного классификатора вместо другого.

### [**Квиз перед лекцией**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/23/)

### **Предварительные знания**

Мы предполагаем, что вы завершили предыдущие уроки, так как будем использовать некоторые концепции, изученные ранее.

Для этого урока нам понадобятся следующие пакеты:

-   `tidyverse`: [tidyverse](https://www.tidyverse.org/) — это [коллекция пакетов для R](https://www.tidyverse.org/packages), разработанная для того, чтобы сделать работу с данными быстрее, проще и интереснее!

-   `tidymodels`: [tidymodels](https://www.tidymodels.org/) — это [фреймворк](https://www.tidymodels.org/packages/), представляющий собой коллекцию пакетов для моделирования и машинного обучения.

-   `themis`: [пакет themis](https://themis.tidymodels.org/) предоставляет дополнительные шаги для работы с несбалансированными данными.

Вы можете установить их следующим образом:

`install.packages(c("tidyverse", "tidymodels", "kernlab", "themis", "ranger", "xgboost", "kknn"))`

Кроме того, приведенный ниже скрипт проверяет, установлены ли у вас необходимые пакеты для выполнения этого модуля, и устанавливает их, если они отсутствуют.


In [ ]:
suppressWarnings(if (!require("pacman"))install.packages("pacman"))

pacman::p_load(tidyverse, tidymodels, themis, kernlab, ranger, xgboost, kknn)

## **1. Карта классификации**

В нашем [предыдущем уроке](https://github.com/microsoft/ML-For-Beginners/tree/main/4-Classification/2-Classifiers-1) мы пытались ответить на вопрос: как выбрать между несколькими моделями? Во многом это зависит от характеристик данных и типа задачи, которую мы хотим решить (например, классификация или регрессия).

Ранее мы узнали о различных вариантах классификации данных, используя шпаргалку от Microsoft. Фреймворк машинного обучения Python, Scikit-learn, предлагает похожую, но более детализированную шпаргалку, которая может помочь сузить выбор оценщиков (другое название классификаторов):

<p >
   <img src="../../images/map.png"
   width="700"/>
   <figcaption></figcaption>


> Совет: [посетите эту карту онлайн](https://scikit-learn.org/stable/tutorial/machine_learning_map/) и следуйте по пути, чтобы прочитать документацию.  
>  
> [Справочный сайт Tidymodels](https://www.tidymodels.org/find/parsnip/#models) также предоставляет отличную документацию о различных типах моделей.

### **План** 🗺️

Эта карта очень полезна, если вы хорошо понимаете свои данные, так как вы можете «пройтись» по её путям к решению:

-   У нас есть >50 образцов

-   Мы хотим предсказать категорию

-   У нас есть размеченные данные

-   У нас меньше 100K образцов

-   ✨ Мы можем выбрать Linear SVC

-   Если это не сработает, так как у нас числовые данные

    -   Мы можем попробовать ✨ KNeighbors Classifier

        -   Если это не сработает, попробуйте ✨ SVC и ✨ Ensemble Classifiers

Это очень полезный маршрут для следования. А теперь давайте приступим к делу, используя [tidymodels](https://www.tidymodels.org/) — фреймворк для моделирования: это последовательная и гибкая коллекция пакетов R, разработанная для поощрения хорошей статистической практики 😊.

## 2. Разделите данные и обработайте несбалансированный набор данных.

Из наших предыдущих уроков мы узнали, что существует набор общих ингредиентов для разных кухонь. Также мы заметили, что распределение количества кухонь было довольно неравномерным.

Мы справимся с этим следующим образом:

-   Удалим самые распространённые ингредиенты, которые создают путаницу между различными кухнями, используя `dplyr::select()`.

-   Используем `recipe`, который предварительно обрабатывает данные, чтобы подготовить их для моделирования, применяя алгоритм `over-sampling`.

Мы уже рассматривали это в предыдущем уроке, так что это должно быть легко 🥳!


In [ ]:
# Load the core Tidyverse and Tidymodels packages
library(tidyverse)
library(tidymodels)

# Load the original cuisines data
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/4-Classification/data/cuisines.csv")

# Drop id column, rice, garlic and ginger from our original data set
df_select <- df %>% 
  select(-c(1, rice, garlic, ginger)) %>%
  # Encode cuisine column as categorical
  mutate(cuisine = factor(cuisine))


# Create data split specification
set.seed(2056)
cuisines_split <- initial_split(data = df_select,
                                strata = cuisine,
                                prop = 0.7)

# Extract the data in each split
cuisines_train <- training(cuisines_split)
cuisines_test <- testing(cuisines_split)

# Display distribution of cuisines in the training set
cuisines_train %>% 
  count(cuisine) %>% 
  arrange(desc(n))

### Работа с несбалансированными данными

Несбалансированные данные часто негативно влияют на производительность модели. Многие модели работают лучше всего, когда количество наблюдений равно, и поэтому испытывают трудности с обработкой несбалансированных данных.

Существует два основных способа работы с несбалансированными наборами данных:

-   добавление наблюдений к меньшинству: `Over-sampling`, например, с использованием алгоритма SMOTE, который синтетически генерирует новые примеры для меньшинства, используя ближайших соседей этих случаев.

-   удаление наблюдений из большинства: `Under-sampling`

В нашем предыдущем уроке мы продемонстрировали, как работать с несбалансированными наборами данных, используя `recipe`. Recipe можно рассматривать как план действий, описывающий, какие шаги следует применить к набору данных, чтобы подготовить его к анализу. В нашем случае мы хотим добиться равномерного распределения количества наших кухонь в `обучающем наборе`. Давайте приступим.


In [ ]:
# Load themis package for dealing with imbalanced data
library(themis)

# Create a recipe for preprocessing training data
cuisines_recipe <- recipe(cuisine ~ ., data = cuisines_train) %>%
  step_smote(cuisine) 

# Print recipe
cuisines_recipe

Теперь мы готовы обучать модели 👩‍💻👨‍💻!

## 3. За пределами моделей мультиномиальной регрессии

В нашем предыдущем уроке мы рассмотрели модели мультиномиальной регрессии. Давайте изучим более гибкие модели для классификации.

### Машины опорных векторов

В контексте классификации `Машины опорных векторов` — это метод машинного обучения, который пытается найти *гиперплоскость*, которая "лучше всего" разделяет классы. Давайте рассмотрим простой пример:

<p >
   <img src="../../images/svm.png"
   width="300"/>
   <figcaption>https://commons.wikimedia.org/w/index.php?curid=22877598</figcaption>


H1~ не разделяет классы. H2~ разделяет, но только с небольшим отступом. H3~ разделяет с максимальным отступом.

#### Линейный классификатор опорных векторов

Кластеризация с использованием опорных векторов (SVC) является частью семейства методов машинного обучения, основанных на опорных векторах. В SVC гиперплоскость выбирается таким образом, чтобы правильно разделить `большинство` обучающих наблюдений, но `может ошибочно классифицировать` некоторые из них. Позволяя некоторым точкам находиться на неправильной стороне, SVM становится более устойчивым к выбросам, что улучшает обобщение на новых данных. Параметр, регулирующий это нарушение, называется `cost` и имеет значение по умолчанию 1 (см. `help("svm_poly")`).

Давайте создадим линейный SVC, установив `degree = 1` в полиномиальной модели SVM.


In [ ]:
# Make a linear SVC specification
svc_linear_spec <- svm_poly(degree = 1) %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svc_linear_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svc_linear_spec)

# Print out workflow
svc_linear_wf

Теперь, когда мы зафиксировали шаги предварительной обработки и спецификацию модели в *workflow*, мы можем приступить к обучению линейного SVC и одновременно оценить результаты. Для оценки производительности давайте создадим набор метрик, который будет учитывать: `accuracy` (точность), `sensitivity` (чувствительность), `Positive Predicted Value` (положительное прогностическое значение) и `F Measure` (F-мера).

> `augment()` добавит столбец(ы) с предсказаниями к предоставленным данным.


In [ ]:
# Train a linear SVC model
svc_linear_fit <- svc_linear_wf %>% 
  fit(data = cuisines_train)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)


# Make predictions and Evaluate model performance
svc_linear_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

#### Машина опорных векторов

Машина опорных векторов (SVM) является расширением классификатора опорных векторов, позволяющим учитывать нелинейную границу между классами. По сути, SVM использует *трюк с ядром* для увеличения пространства признаков, чтобы адаптироваться к нелинейным взаимосвязям между классами. Одной из популярных и чрезвычайно гибких функций ядра, используемых в SVM, является *радиальная базисная функция*. Давайте посмотрим, как она справится с нашими данными.


In [ ]:
set.seed(2056)

# Make an RBF SVM specification
svm_rbf_spec <- svm_rbf() %>% 
  set_engine("kernlab") %>% 
  set_mode("classification")

# Bundle specification and recipe into a worklow
svm_rbf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(svm_rbf_spec)


# Train an RBF model
svm_rbf_fit <- svm_rbf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
svm_rbf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Намного лучше 🤩!

> ✅ Пожалуйста, ознакомьтесь:
>
> -   [*Support Vector Machines*](https://bradleyboehmke.github.io/HOML/svm.html), Практическое машинное обучение с использованием R
>
> -   [*Support Vector Machines*](https://www.statlearning.com/), Введение в статистическое обучение с приложениями на R
>
> для дополнительного чтения.

### Классификаторы ближайших соседей

Алгоритм *K*-ближайших соседей (KNN) предсказывает каждое наблюдение на основе его *сходства* с другими наблюдениями.

Давайте применим его к нашим данным.


In [ ]:
# Make a KNN specification
knn_spec <- nearest_neighbor() %>% 
  set_engine("kknn") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
knn_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(knn_spec)

# Train a boosted tree model
knn_wf_fit <- knn_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
knn_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Кажется, что эта модель работает не очень хорошо. Вероятно, изменение аргументов модели (см. `help("nearest_neighbor")`) улучшит её производительность. Обязательно попробуйте это сделать.

> ✅ Пожалуйста, ознакомьтесь:
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> чтобы узнать больше о классификаторах *K*-ближайших соседей.

### Ансамблевые классификаторы

Ансамблевые алгоритмы работают, объединяя несколько базовых оценок для создания оптимальной модели, используя один из подходов:

`bagging`: применение *усредняющей функции* к набору базовых моделей

`boosting`: построение последовательности моделей, которые дополняют друг друга для улучшения предсказательной производительности.

Давайте начнем с модели случайного леса (Random Forest), которая создает большое количество деревьев решений, а затем применяет усредняющую функцию для получения более качественной общей модели.


In [ ]:
# Make a random forest specification
rf_spec <- rand_forest() %>% 
  set_engine("ranger") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
rf_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(rf_spec)

# Train a random forest model
rf_wf_fit <- rf_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
rf_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

Отличная работа 👏!

Давайте также поэкспериментируем с моделью Boosted Tree.

Boosted Tree определяет ансамблевый метод, который создает серию последовательных деревьев решений, где каждое дерево зависит от результатов предыдущих деревьев, пытаясь постепенно уменьшить ошибку. Этот метод фокусируется на весах неправильно классифицированных элементов и корректирует подгонку для следующего классификатора, чтобы исправить ошибки.

Существует несколько способов подгонки этой модели (см. `help("boost_tree")`). В этом примере мы будем подгонять Boosted Trees с использованием движка `xgboost`.


In [ ]:
# Make a boosted tree specification
boost_spec <- boost_tree(trees = 200) %>% 
  set_engine("xgboost") %>% 
  set_mode("classification")

# Bundle recipe and model specification into a workflow
boost_wf <- workflow() %>% 
  add_recipe(cuisines_recipe) %>% 
  add_model(boost_spec)

# Train a boosted tree model
boost_wf_fit <- boost_wf %>% 
  fit(data = cuisines_train)


# Make predictions and Evaluate model performance
boost_wf_fit %>% 
  augment(new_data = cuisines_test) %>% 
  eval_metrics(truth = cuisine, estimate = .pred_class)

> ✅ Пожалуйста, ознакомьтесь:
>
> -   [Machine Learning for Social Scientists](https://cimentadaj.github.io/ml_socsci/tree-based-methods.html#random-forests)
>
> -   [Hands-on Machine Learning with R](https://bradleyboehmke.github.io/HOML/)
>
> -   [An Introduction to Statistical Learning with Applications in R](https://www.statlearning.com/)
>
> -   <https://algotech.netlify.app/blog/xgboost/> - Рассматривается модель AdaBoost, которая является хорошей альтернативой xgboost.
>
> чтобы узнать больше о ансамблевых классификаторах.

## 4. Дополнительно - сравнение нескольких моделей

Мы обучили довольно много моделей в этой лабораторной работе 🙌. Создание множества рабочих процессов с различными наборами препроцессоров и/или спецификаций моделей, а затем расчет метрик производительности по одной может стать утомительным или обременительным.

Давайте посмотрим, сможем ли мы решить эту задачу, создав функцию, которая обучает список рабочих процессов на обучающем наборе, а затем возвращает метрики производительности на основе тестового набора. Мы будем использовать `map()` и `map_dfr()` из пакета [purrr](https://purrr.tidyverse.org/), чтобы применять функции к каждому элементу списка.

> Функции [`map()`](https://purrr.tidyverse.org/reference/map.html) позволяют заменить множество циклов for на код, который более лаконичен и легче читается. Лучшее место для изучения функций [`map()`](https://purrr.tidyverse.org/reference/map.html) — это [глава об итерациях](http://r4ds.had.co.nz/iteration.html) в книге "R for Data Science".


In [ ]:
set.seed(2056)

# Create a metric set
eval_metrics <- metric_set(ppv, sens, accuracy, f_meas)

# Define a function that returns performance metrics
compare_models <- function(workflow_list, train_set, test_set){
  
  suppressWarnings(
    # Fit each model to the train_set
    map(workflow_list, fit, data = train_set) %>% 
    # Make predictions on the test set
      map_dfr(augment, new_data = test_set, .id = "model") %>%
    # Select desired columns
      select(model, cuisine, .pred_class) %>% 
    # Evaluate model performance
      group_by(model) %>% 
      eval_metrics(truth = cuisine, estimate = .pred_class) %>% 
      ungroup()
  )
  
} # End of function

In [ ]:
# Make a list of workflows
workflow_list <- list(
  "svc" = svc_linear_wf,
  "svm" = svm_rbf_wf,
  "knn" = knn_wf,
  "random_forest" = rf_wf,
  "xgboost" = boost_wf)

# Call the function
set.seed(2056)
perf_metrics <- compare_models(workflow_list = workflow_list, train_set = cuisines_train, test_set = cuisines_test)

# Print out performance metrics
perf_metrics %>% 
  group_by(.metric) %>% 
  arrange(desc(.estimate)) %>% 
  slice_head(n=7)

# Compare accuracy
perf_metrics %>% 
  filter(.metric == "accuracy") %>% 
  arrange(desc(.estimate))


[**workflowset**](https://workflowsets.tidymodels.org/) пакет позволяет пользователям создавать и легко подбирать большое количество моделей, но в основном предназначен для работы с методами повторной выборки, такими как `кросс-валидация`, подход, который мы еще не рассматривали.

## **🚀Задача**

Каждая из этих техник имеет множество параметров, которые можно настроить, например, `cost` в SVM, `neighbors` в KNN, `mtry` (случайно выбранные предикторы) в Random Forest.

Изучите параметры по умолчанию для каждой из них и подумайте, что изменение этих параметров может означать для качества модели.

Чтобы узнать больше о конкретной модели и ее параметрах, используйте: `help("model")`, например, `help("rand_forest")`.

> На практике мы обычно *оцениваем* *лучшие значения* для этих параметров, обучая множество моделей на `смоделированном наборе данных` и измеряя, насколько хорошо эти модели работают. Этот процесс называется **тюнингом**.

### [**Викторина после лекции**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/24/)

### **Обзор и самостоятельное изучение**

В этих уроках много терминологии, поэтому уделите минуту, чтобы просмотреть [этот список](https://docs.microsoft.com/dotnet/machine-learning/resources/glossary?WT.mc_id=academic-77952-leestott) полезных терминов!

#### СПАСИБО:

[`Эллисон Хорст`](https://twitter.com/allison_horst/) за создание потрясающих иллюстраций, которые делают R более доступным и увлекательным. Найдите больше иллюстраций в ее [галерее](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

[Кэсси Бревью](https://www.twitter.com/cassieview) и [Джен Лупер](https://www.twitter.com/jenlooper) за создание оригинальной версии этого модуля на Python ♥️

Приятного обучения,

[Эрик](https://twitter.com/ericntay), Золотой Студенческий Посол Microsoft Learn.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="569"/>
   <figcaption>Иллюстрация от @allison_horst</figcaption>



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его исходном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникшие в результате использования данного перевода.
